<a href="https://colab.research.google.com/github/othoni-hub/NSI/blob/main/TP_Geolocalisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP Python : TRAITEMENT DE DONNÉES GÉOLOCALISÉES

## Olivier THÖNI

## FACULTÉ d'ÉDUCATION UCO - DU ES.NSI

*document sous licence Creative Commons - CC : By/NC/SA*




In [1]:
# -*- coding: utf-8 -*-

Le travail en Python sur données réelles ouvertes permet de belles activités d'apprentissage.

## 1. La Station Spatiale Internationale (ISS)

*Sources :*
* Valentin Balivet, Mathieu Soul : https://fabdev.fr/articles/iss-python/


Petit TP pour prendre en mains quelques modules de Python :
* **requests** : permet d'aller chercher des informations en ligne
* **folium** : permet de positionner des points sur une carte géographique
* **time** : contrôle du temps (accessoirement)



Nous allons pouvoir suivre le parcours de l'ISS en temps réel, à 8 km/s !

Tout d'abord, envoyons, grâce au module "requests", une requête à l'application "People in Space" pour obtenir les coordonnées géographiques actuelles de la station :

In [2]:
import requests
json_reponse = requests.get('http://api.open-notify.org/iss-now.json').json()
print(json_reponse)

{'message': 'success', 'timestamp': 1632407324, 'iss_position': {'latitude': '-50.0402', 'longitude': '-74.6636'}}


Le résultat est un fichier de type "JSON", contenant les informations de localisation de l'ISS au moment où la requête a été passée.

In [3]:
latitude = json_reponse['iss_position']['latitude']
longitude = json_reponse['iss_position']['longitude']

Grâce à la librairie "folium", plaçons sur une carte OpenStreetMap définie par son centre et son zoom, grâce à la commande ".Map", ce point sous forme d'un "Marker" :

In [4]:
import folium

coordonnees = [float(latitude), float(longitude)]
carte = folium.Map(coordonnees, tiles = "OpenStreetMap", zoom_start=8)
folium.Marker([float(latitude), float(longitude)]).add_to(carte)
carte

Si rien ne s'affiche, utiliser le module "webbrowser" pour l'afficher dans son navigateur

In [5]:
#import webbrowser
#filepath = 'C:/Users/Olivier/Desktop/map.html'
#carte.save(filepath)
#webbrowser.open('file://' + filepath)

Reste à lancer la requête à intervalles réguliers, grâce au module "time"

In [6]:
import time

points = []
nb_points = 20
tempo = 6
for i in range(nb_points) :
    position = requests.get('http://api.open-notify.org/iss-now.json').json()
    
    latitude = position['iss_position']['latitude']
    longitude = position['iss_position']['longitude']
    
    coordonnees = [float(latitude), float(longitude)]
    points.append(coordonnees)
    
    carte = folium.Map(coordonnees, tiles = "OpenStreetMap", zoom_start=5)
    
    time.sleep(tempo)
    
folium.Marker([float(latitude), float(longitude)]).add_to(carte) 
folium.PolyLine(points).add_to(carte) # permet de tracer la ligne entre les points repérés.
carte


In [7]:
#import os
#os.remove('iss-now.json') 
# on élimine le fichier de données pour faire la place pour le prochain lancement de ce bloc 

## 2. Où est donc passé mon bus ?

*source : **David Cobac**.*


### 2.1. Téléchargement des données ouvertes


On va travailler ici avec un fichier **"csv"**, mis à jour en temps réel (chaque minute), donnant la position de tous les bus en circulation dans Angers.


Pour cela, nous allons utiliser le package **"wget"**, et, bien sûr, le package **"folium"** pour la géolocalisation.



In [9]:
!pip install wget
import wget
url = "https://data.angers.fr/explore/dataset/bus-tram-position-tr/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true"
fichier_bus = wget.download(url, 'donnees_bus.csv')


  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=d876484e0d851b293809e25b56203ed262064ce5f2e13bca2669e47c7e2add14
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


### 2.2. Visualisation des bus en temps réel

In [10]:
# le fichier téléchargé est un fichier csv avec séparateur de champs ";", on l'éclate pour récupérer les différents champs :
import folium 

latitude_ville = 47.48
longitude_ville = -0.49   # Angers
carte = folium.Map(location = [latitude_ville, longitude_ville], tiles = "OpenStreetMap", zoom_start = 12)

with open('donnees_bus.csv') as fh :
    noms = fh.readline().split(";")
    indice = noms.index('coordonnees')
    #indice2 = noms.index('mnemoligne')

    for ligne in fh :
        #print(ligne)
        couple_gps = ligne.split(";")[indice] # couple_gps est de type "str"
        lat, long = couple_gps.split(",")
        
        latitude = float(lat)
        longitude = float(long)
        
        coord = [latitude, longitude]
        #marque = folium.Marker([latitude, longitude]) # ajoute un marqueur classique
        carte.add_child(folium.RegularPolygonMarker(location=[latitude, longitude],
                       fill_color='#132b5e', radius=3))
        #marque.add_to(carte)
        
        #num_ligne = ligne.split(";")[indice2]
        #if num_ligne == '03':
            #points.append(coord)
            
            
#folium.PolyLine(points).add_to(carte) # permet de tracer la ligne entre les points repérés.            
carte


### 2.3. Visualisation et traitement des données avec Pandas

In [11]:
import pandas as pd
bus_tous = pd.read_csv('donnees_bus.csv', encoding='latin', delimiter=";")  

bus_tous

,idvh,novh,type,etat,idligne,iddesserte,x,y,cap,coordonnees,sv,idarret,harret,ecart,ts_maj,mnemoligne,nomligne,mnemoarret,nomarret,numarret,idparcours,dest
0,268435913,457,OMNICITY,LIGN,268435468,269231389,382785,2278192,9,"47.4675,-0.54547462",1204,2931105,2021-09-23T16:37:31+02:00,229,2021-09-23T16:37:30+02:00,12,M-MARCILLE <> ST AUBIN LA SALLE,HMAI,HUIT MAI LORRAINE,11449,269231360,SAINT AUBIN LA SALLE
1,268436465,1009,CITADIS,TDEP,268435471,269455361,379167,2283215,136,"47.511446,-0.59591985",A14,2930605,2021-09-23T16:40:20+02:00,0,2021-09-23T16:36:54+02:00,A,ARDENNE <> ROSERAIE,1AARD,AVRILLE - ARDENNE,5009,269455360,ANGERS ROSERAIE
2,268444266,8810,CROSSWAY,TDEP,268435491,270736129,381721,2277967,347,"47.465126,-0.559469",3511,2932003,2021-09-23T16:40:00+02:00,0,2021-09-23T16:37:12+02:00,35,ST LEGER St LAMBERT <> GARE,GARESEMA,GARES SEMARD,21525,270736128,SAINT LEGER LA COUDRE
3,268435896,440,OMNICITY,TARR,268435467,269164332,377265,2278329,60,"47.466888,-0.61871387",1103,2932437,2021-09-23T16:31:52+02:00,0,2021-09-23T16:33:33+02:00,11,LAC MAINE <> STE GEMMES CL ANJOU,LHOIRI-E,L'HOIRIE,7844,269164288,L'HOIRIE LAC DE MAINE
4,268435904,448,OMNICITY,LIGN,268435465,269068818,382764,2278768,207,"47.472671,-0.54603322",0904,2931299,2021-09-23T16:37:17+02:00,235,2021-09-23T16:38:56+02:00,09,ESPACE ANJOU <> EVENTARD,MENDESFR,MENDES FRANCE,10353,269068800,ESPACE ANJOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,268436278,822,CITYSTD,LIGN,268435463,268903183,379859,2281209,321,"47.493647,-0.58574909",0724,2930789,2021-09-23T16:37:34+02:00,101,2021-09-23T16:38:56+02:00,07,HOPITAL <> MONTREUIL JUIGNE,BRASSE-E,BRASSENS,25269,268903168,FENEU par MONTREUIL JUIGN
148,268436461,1005,CITADIS,LIGN,268435471,269454361,381448,2276022,129,"47.447552,-0.56213624",A11,2930620,2021-09-23T16:38:27+02:00,517,2021-09-23T16:38:56+02:00,A,ARDENNE <> ROSERAIE,1JVIL,JEAN VILAR,20509,269454336,ANGERS ROSERAIE
149,268436467,1011,CITADIS,LIGN,268435471,269422858,382707,2278656,24,"47.471645,-0.54673438",A06,2932507,2021-09-23T16:37:09+02:00,158,2021-09-23T16:38:56+02:00,A,ARDENNE <> ROSERAIE,2HDVA,HOTEL DE VILLE,184082,269422848,AVRILLE ARDENNE
150,268438708,3252,OTOKAR,LIGN,268435493,270871299,381460,2277235,220,"47.45846,-0.56257051",3707,2932234,2021-09-23T16:40:42+02:00,242,2021-09-23T16:38:56+02:00,37,SAVENNIERES <> GARE,GAUBOU-E,GAUBOURGS,23234,270871296,SAVENNIERES


In [12]:
print("{} bus roulant !".format(len(bus_tous)))

152 bus roulant !


In [13]:
ligne_03 = bus_tous[bus_tous['mnemoligne'].str.match("03")] # DatraFrame
print("{} bus roulant ligne n°3 !".format(len(ligne_03)))
ligne_03

11 bus roulant ligne n°3 !


,idvh,novh,type,etat,idligne,iddesserte,x,y,cap,coordonnees,sv,idarret,harret,ecart,ts_maj,mnemoligne,nomligne,mnemoarret,nomarret,numarret,idparcours,dest
11,268436150,694,CITYART,LIGN,268435459,268644618,384066,2270630,63,"47.399947,-0.52484157",0302,2930722,2021-09-23T16:37:19+02:00,-147,2021-09-23T16:38:56+02:00,03,MURS ERIGNE <> ADEZIERE SALETTE,BARBOTER,BARBOTTERIE,24521,268644608,AVRILLE ADEZIERE
25,268436145,689,CITYART,TARR,268435459,268644667,378480,2282515,270,"47.504924,-0.60468626",0324,2932434,2021-09-23T16:25:38+02:00,0,2021-09-23T16:27:33+02:00,03,MURS ERIGNE <> ADEZIERE SALETTE,DEZIE AR,AVRILLE ADEZIERE,19310,268644608,AVRILLE ADEZIERE
36,268436062,606,MAN City,LIGN,268435459,268676624,379678,2280387,124,"47.486198,-0.58774368",0301,2930895,2021-09-23T16:37:30+02:00,-29,2021-09-23T16:38:56+02:00,03,MURS ERIGNE <> ADEZIERE SALETTE,CLINIQUE,CLINIQUE ST DIDIER,26512,268676608,MURS ERIGNE
42,268436168,712,City_GNV,LIGN,268435459,268644639,381927,2278269,302,"47.467909,-0.55688586",0303,2930828,2021-09-23T16:37:29+02:00,-156,2021-09-23T16:38:56+02:00,03,MURS ERIGNE <> ADEZIERE SALETTE,CCICHA-E,CCI,24375,268644608,AVRILLE ADEZIERE
59,268436151,695,CITYART,TDEP,268435459,268644609,381719,2270211,61,"47.395408,-0.55570975",0321,2931351,2021-09-23T16:46:00+02:00,0,2021-09-23T16:35:35+02:00,03,MURS ERIGNE <> ADEZIERE SALETTE,MURSERIG,MURS ERIGNE,22074,268644608,AVRILLE ADEZIERE
69,268436068,612,MAN City,LIGN,268435459,268644663,379178,2282538,0,"47.505365,-0.59543864",0325,2930714,2021-09-23T16:37:09+02:00,2,2021-09-23T16:38:56+02:00,03,MURS ERIGNE <> ADEZIERE SALETTE,BALAND-E,BASSE LANDE,30198,268644608,AVRILLE ADEZIERE
107,268436075,619,MAN City,LIGN,268435459,268676636,382044,2278460,243,"47.469664,-0.55542812",0323,2931362,2021-09-23T16:38:19+02:00,-118,2021-09-23T16:38:56+02:00,03,MURS ERIGNE <> ADEZIERE SALETTE,OFFTOU-E,OFFICE DE TOURISME,3776,268676608,MURS ERIGNE
130,268436064,608,MAN City,LIGN,268435459,268644644,381784,2278913,315,"47.47365,-0.5590962",0305,2930735,2021-09-23T16:37:21+02:00,322,2021-09-23T16:38:56+02:00,03,MURS ERIGNE <> ADEZIERE SALETTE,BEAURE-E,BEAUREPAIRE,6393,268644608,AVRILLE ADEZIERE
131,268436066,610,MAN City,LIGN,268435459,268644627,383916,2274133,348,"47.431386,-0.52851919",0304,2930705,2021-09-23T16:37:11+02:00,208,2021-09-23T16:38:56+02:00,03,MURS ERIGNE <> ADEZIERE SALETTE,AUTHION,AUTHION,22482,268644608,AVRILLE ADEZIERE
133,268436078,622,MAN City,LIGN,268435459,268676640,382656,2277532,158,"47.461525,-0.54686347",0322,2931461,2021-09-23T16:37:05+02:00,142,2021-09-23T16:38:56+02:00,03,MURS ERIGNE <> ADEZIERE SALETTE,RABELA-E,RABELAIS,28844,268676608,MURS ERIGNE


### 2.4. Nettoyage

(sinon, le fichier ne se remet pas à jour :  si le fichier existe déjà, l'explorateur change son nom au téléchargement en ajoutant un numéro de version, ce qui le rend inaccessible à ce programme Python.

In [16]:
import os
os.remove('donnees_bus.csv') # on élimine le fichier de données pour faire la place pou le prochain lancement de ce bloc 

**prolongement possible :**

Ajouter une boucle mettant à jour le fichier chaque minute et générant un fichier html ?

# 3. Carte "choroplèthe" : nombre d'habitants dans les communes du Maine-et-Loire
*Source : **David Cobac**

Une carte choroplète est une représentation graphiques de données statistiques sur une carte géographique.

La carte est découpée en polygones (frontières administratives des pays, des départements des communes, etc.), colorés sur une échelle de teintes en fonction de la valeur moyenne du caractère étudié.

Les données cartographiques sont importées au format **"json"**, les données statistiques, dans un fichier **"csv"**

Observons, par exemple, la population des communes du Maine-et-Loire


Sur https://api.gouv.fr/api/api-geo.html , on paramètre la géométrie recherchée : ici le département 49
Voir aussi https://france-geojson.gregoiredavid.fr/ 

In [17]:
import pandas as pd
import json

latitude_ville = 47.48
longitude_ville = -0.49   # Angers
   
#import wget
#url = "https://geo.api.gouv.fr/departements/49/communes?fields=nom,code,codesPostaux,codeDepartement,codeRegion,population&format=json&geometry=centre"
#geometrie = wget.download(url)

formes = 'aura_pop_90_99_07_12csv.geojson' #"formes" : chemin du fichier à ouvrir, ici présent dans le même répertoire que le présent NoteBook

with open(formes) as fh_formes :
    fformes = json.load(fh_formes)
#tab_formes_annees = []
#indice = 0
#for f in fformes["features"]:
#    tab_formes_annees.append(f[indice]["properties"]["CODGEO"])
    

FileNotFoundError: ignored

Récupération données population par commune

In [ ]:
donnees_csv = "aura_pop_90_99_07_12csv.csv"

df = pd.read_csv(donnees_csv, sep=";")# "df" comme DataFrame
series = df[['CODGEO', 'P12_POP']]
series=series.astype({'CODGEO': int, 'P12_POP': int})

series.rename(columns={'CODGEO': "codgeo"}, inplace=True)

In [ ]:
#df.head()
df

,CODGEO,LibCommune,D90_POP,D99_POP,P07_POP,P12_POP,Geo Shape,geo_point_2d
0,49007,ANGERS,141404,151279,151108,149017,"{""type"": ""Polygon"", ""coordinates"": [[[-0.56152...","47.4766540244, -0.556233092842"
1,49015,AVRILLE,12878,12991,12538,12953,"{""type"": ""Polygon"", ""coordinates"": [[[-0.58156...","47.505300955, -0.600602806852"
2,49035,BOUCHEMAINE,5799,6153,5886,6459,"{""type"": ""Polygon"", ""coordinates"": [[[-0.59528...","47.4307335803, -0.627406600476"
3,49048,BRIOLLAY,2005,2282,2565,2751,"{""type"": ""Polygon"", ""coordinates"": [[[-0.50107...","47.5763756202, -0.498140700622"
4,49129,ECOUFLANT,3361,3703,3747,3775,"{""type"": ""Polygon"", ""coordinates"": [[[-0.53178...","47.5285191826, -0.518355800323"
5,49130,ECUILLE,380,457,571,617,"{""type"": ""Polygon"", ""coordinates"": [[[-0.54419...","47.6153233473, -0.561081386288"
6,49223,MURS-ERIGNE,4224,5115,5305,5370,"{""type"": ""Polygon"", ""coordinates"": [[[-0.54580...","47.394959861, -0.553477933524"
7,49298,SAINT LEGER DES BOIS,978,1307,1491,1563,"{""type"": ""Polygon"", ""coordinates"": [[[-0.69847...","47.462635906, -0.727435864058"
8,49306,SAINT MARTIN DU FOUILLOUX,1315,1368,1586,1651,"{""type"": ""Polygon"", ""coordinates"": [[[-0.67152...","47.4369488198, -0.708103095622"
9,49241,LE PLESSIS-GRAMMOIRE,1718,2013,2219,2295,"{""type"": ""Polygon"", ""coordinates"": [[[-0.43385...","47.4921747062, -0.436429706455"


In [ ]:
print(series)

    codgeo  P12_POP
0    49007   149017
1    49015    12953
2    49035     6459
3    49048     2751
4    49129     3775
5    49130      617
6    49223     5370
7    49298     1563
8    49306     1651
9    49241     2295
10   49242     1256
11   49278     3648
12   49326      816
13   49339     1455
14   49055     2047
15   49196     2175
16   49267     8837
17   49271     2049
18   49329     1393
19   49353    12929
20   49200     2012
21   49214     7174
22   49294     2541
23   49338     1194
24   49020     4889
25   49028      124
26   49135     2172
27   49238     2499
28   49246    11975
29   49251      693
30   49289     1779
31   49323     4458
32   49337     2609
33   49377     2858


Création de la carte choroplèthe :

In [ ]:
import folium

carte = folium.Map(location = [latitude_ville, longitude_ville], tiles = "OpenStreetMap", zoom_start = 11)


classes = [0]
classes += [1000 * 3 ** i for i in range(6)]
#classes = [100,50000,100000,3000000]

folium.Choropleth(geo_data = fformes,
                  data = series,
                  bins = classes,
                  columns=["codgeo", "P12_POP"],
                  key_on="feature.properties.codgeo",
                  fill_color="PuRd", 
                  fill_opacity=0.6,
                  line_opacity=0.9,
                  legend_name='Population'

).add_to(carte) #key_on="feature.properties.CODGEO",

# Accorder la plus grande attention à la propriété "key_on"
# C'est elle qui permet la jointure entre les deux tables
# celle au format "json" et celle au format "pandas" :
# elles doivent porter le même nom et comporter le même
# nombre de données dans le même format !!!

carte

In [ ]:
classes

[0, 1000, 3000, 9000, 27000, 81000, 243000]

In [ ]:
# fill_color: vous pouvez choisir l'une des palettes suivantes: 
#"BuGn", "BuPu", "GnBu", "OrRd", "PuBu", "PuBuGn", "PuRd", "RdPu", 
#"YlGn", "YlGnBu" , 'YlOrBr' et 'YlOrRd'.

In [ ]:
#fformes